In [1]:
import os
import glob
import pprint
import re
import typing
from itertools import cycle
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 2000)

os.chdir("/workspace/pepsico")

/tmp/ipykernel_118486/3382551506.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
REG_BLOCK_COMMENT = re.compile("(/\*)[\w\W]*?(\*/)", re.IGNORECASE)
REG_LINE_COMMENT = re.compile('(--.*)', re.IGNORECASE)

In [3]:
def split_table_info(table_info: str):
    table_info = table_info.split(".")
    #print('table_info', table_info)
    if len(table_info) >= 3:
        table_name = table_info[-1]
        table_schema_name = table_info[-2]
    elif len(table_info) == 2:
        table_name = table_info[1]
        table_schema_name = table_info[0]
    else:
        table_name = table_info[0]
        table_schema_name = ''
    
    return table_schema_name, table_name

t = "ACQ_O.SN_CI_VW"
a, b =  split_table_info(t)
print(a,'-', b)

ACQ_O - SN_CI_VW


In [24]:
def create_view_name(block : str):
    
    metadata = []
    for s in block.split('\n'):
        if re.search("^CREATE OR REPLACE",s):
            schema_proc = s.strip().split(' ')[-1]
            print('schema_proc', schema_proc)
            Schema, ViewName = split_table_info(schema_proc)
            metadata.append([Schema, ViewName])
        else:
            pass
        

    return metadata

create_view_name(f"""CREATE OR REPLACE FORCE VIEW ACQ_O.SN_CI_VW
(
    CI_SYS_ID,
    CI_NM,
    SHORT_DESCRIPTION,
    SYS_CREATED_ON,
    CLASS_NAME,
    LOC_NM,
    ASSET_TAG,
    MODEL_ID,
    MODEL_NUM,
    SERIAL_NUM,
    INSTALL_STATUS,
    SUPPORT_GROUP_SYS_ID,
    SUPPORT_GROUP,
    DISCOVERY_SOURCE,
    FIRST_DISCOVERED,
    LAST_DISCOVERED,
    CATEGORY,""")


schema_proc ACQ_O.SN_CI_VW


[['ACQ_O', 'SN_CI_VW']]

In [25]:
REG_BLOCK_COMMENT = re.compile("(/\*)[\w\W]*?(\*/)", re.IGNORECASE)
REG_LINE_COMMENT = re.compile('(--.*)', re.IGNORECASE)

def code_block_parsing(block: str) -> list:
    
    metadata = []
    
    for s in block.split('\n'): 
        
        # Line Comments
        if re.search("(--.*)", s):
            s = s.replace(s,'')
        
        # Drop View
        if re.search("\s*DROP VIEW", s):
            Operation = "Drop view"
            table_info = s.strip().replace(";","").split(' ')[-1]
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        # Grant access
        if re.search("\s*SELECT ON\s+", s):
            table_info = s.strip().replace(";","").split(' ')[3]
            user_group = s.strip().replace(";","").split(' ')[-1]
            Operation = f"Grant read access to {user_group}"
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
            
        if re.search("\s*INSERT INTO\s",s):
            Operation = 'Insert into'
            table_info = s.strip().split("INSERT INTO")[-1].split()[0]  # Split 'INSERT INTO'
            #print('insert into table:', table_info)
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        if re.search("\s*UPDATE\s",s):
            Operation = 'Update'
            table_info = s.strip().split(' ')[1]   # split 'UPDATE'
            #print('update table info:', table_info)
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        if re.search("\s*DELETE FROM\s",s):
            Operation = 'Delete'
            table_info = s.strip().split(' ')[2]   # split 'DELETE FROM'
            #print('delete from table:', table_info)
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        if re.search("\s*MERGE INTO\s",s):
            Operation = 'Merge into'
            table_info = s.strip().split(' ')[2]   # split 'MERGE'
            #print('merge into table:', table_info)
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        if re.search("\s*CURSOR\s",s):
            Operation = 'Cursor'
            table_info = s.strip().split(' ')[1]   # split 'CURSOR'
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        # TRUNCATE TABLE
        if re.search("\s*TRUNCATE\s+TABLE\s+", s):
            Operation = 'Truncate table'
            table_info = s.strip().split('TRUNCATE TABLE')[-1].replace("'"," ").split()[0]   # split 'CURSOR'
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
        
        # FROM commands
        if re.search("^(?!.*(?:DELETE))(?=\s*(?:\w+FROM_\w+))", s):
            pass
        
        if re.search("^(?!.*(?:DELETE))(?=.*(?:FROM)\s+(\w+))", s):
            #print('\n----------------------')
            #print('s:',s)
            Operation = 'Select'
            table_info = s.strip().split('FROM')[-1].split()[0].replace(")"," ").split()[0]     
            #print(f'select from :{table_info}')
            Table_Schema_Name, Table = split_table_info(table_info)
            #print(f'Table_Schema_Name:{Table_Schema_Name}\n Table:{Table}')
            #print('\n----------------------\n')
            metadata.append([Operation, Table_Schema_Name, Table])
            
        if re.search("^(?!.*(?:DELETE))(?=.*(?:FROM$))", s):
            pass
        
        # JOIN commands
        if re.search("\s*JOIN\s",s):
            Operation = 'Join'
            table_info = s.strip().split('JOIN')[-1].split()[0]   # split 'CURSOR'
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])
            
        if re.search("\s.*JOIN\s+.SELECT",s):
            Operation = 'Join'
            table_info = s.strip().split('JOIN')[-1].split('FROM')[-1].split()[0].replace(")", " ").split()[0]  # split 'CURSOR'
            Table_Schema_Name, Table = split_table_info(table_info)
            metadata.append([Operation, Table_Schema_Name, Table])

    return metadata

In [44]:
os.chdir("/workspace/pepsico")

df = pd.DataFrame(columns = ['File', 'View_No','Schema', 'View_Name', 'Operation_No',
                             'Operation', 'Table_Schema_Name','Tables', 'Sequence'])

for filename in glob.glob('*.sql'):
    fname = filename.replace(".sql","")
    print(f'Filename: {fname}\n')
    file = open(filename, 'r')
    sqlFile = file.read()
    file.close()
    lines = re.split(";\n", sqlFile)
    lines = [line for line in lines if line != "\n\n\n" if line != "\n\n" if line != ""]
    d1 = []
    for line in lines:
        m = create_view_name(line)
        #print(m)
        if len(m) != 0:
            #print(f'schema0:{m[0][0]} - procedure0:{m[0][1]}\n')
            result = [[m[0][0]]+[m[0][1]]+i for i in code_block_parsing(line)]
        else:
            result = [['']+['']+i for i in code_block_parsing(line)]
        d1.extend(result)
        #print(d1)
        
        
    d2 = pd.DataFrame(d1, columns = ['Schema', 'View_Name', 'Operation', 'Table_Schema_Name','Tables'])
    d2.insert(0, 'File', filename)
    d2 = d2[~(d2.Tables.isin(["SET", "UPDATE", "(SELECT", "_DT,"]))]
    d2['Sequence'] = d2.reset_index().index+1
    df = pd.concat([d2])
    #print('---------------------------')
        
        
    
    

Filename: DWL_DATA_O.TBM_SN_BUSN_APP_VW

schema_proc DWL_DATA_O.TBM_SN_BUSN_APP_VW


In [45]:
df

,File,Schema,View_Name,Operation,Table_Schema_Name,Tables,Sequence
0,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,,,Drop view,DWL_DATA_O,TBM_SN_BUSN_APP_VW,1
1,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Select,ACQ_O,CMDB_CI_BUSINESS_APP,2
2,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,DWL_DATA_O,SN_CI_CLSS_VW,3
3,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,,SN_CNTCT_VW,4
4,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Select,ACQ_O,SYS_CHOICE,5
5,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Select,,SN_CI_BUSN_APP_CPBLTY_XREF_VW,6
6,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,,SN_CI_BUSN_CPBLTY_HRCHY_VW,7
7,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,DWL_DATA_O,SN_APM_TRM_DCLRTN_PRDCT,8
8,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,DWL_DATA_O,SN_CI_APP_EA_DCLRTN_TYP,9
9,DWL_DATA_O.TBM_SN_BUSN_APP_VW.sql,DWL_DATA_O,TBM_SN_BUSN_APP_VW,Join,DWL_DATA_O,SN_CNTCT,10
